<div style="text-align: center;">
    <span style="color: red; font-size: 70px;"> Monte-Carlo paths</span>
</div>

Un sistema quantistico è governato dalla matrice densità, che è definita per mezzo delle funzioni d'onda e degli autostati di energia. Il calcolo analitico della matrice densità non è scontato, in quanto non sempre conosciamo esplicitamente le soluzioni dell'equazione di Schrodinger.

<center>

#### Tecnica di convoluzione
</center>

La matrice densità presenta un'espressione semplice solo ad alte temperature: la casistica a bassa temperatura può essere recuperata dal limite $T \gg 1$ mediante una tecnica di convoluzione, che applicata in modo iterativo pone le basi per l'integrale sui cammini di Feynman.

$$
\int dx' \rho\left(x,\,x',\,\beta_1\right) \rho\left(x',\,x'',\,\beta_2\right)\,=\,\rho\left(x,\,x'',\,\beta_1\,+\,\beta_2\right)
$$

<center>

#### Feynman path integral
</center>

La convoluzione di matrici densità ad alte temperature non può essere sempre valutata in modo analitico, ma è possibile ottenerne una stima per mezzo di tecniche Monte-Carlo. Procediamo in modo iterativo esplicitando tutte le integrazioni contemporaneamente, in modo tale che al passo k-esimo il numero di matrici densità considerate sia $2^k$:

$$
\rho\left(x_0,\,x_N,\,\beta\right)\,=\,\int dx_1 \dots dx_{N-1}\,\rho\left(x_0,\,x_1,\,\beta/N\right)\dots\rho\left(x_{N-1},\,x_N,\,\beta/N\right)
$$

La funzione di partizione, che è la traccia della matrice densità, viene valutata andando ad introdure un'ulteriore integrazione, in modo tale che:

$$
Z\,=\,\int dx_0\,\rho\left(x_0,\,x_0,\,\beta\right)
$$

La configurazione $\left\{x_0,\,\dots,\,x_N\right\}$ è detta cammino, della qual possiamo interpretare la variabile $x_k$ come la posizione al tempo d'evoluzione immaginario $k\beta/N$. Nel momento in cui il numero di completezze introdotte in precedenza è elevato, è possibile lavorare con la decomposizione di Trotter, trascurando il fatto che $\left[H,\,V\right]\,\neq\, 0$, ottenendo che:

$$
\exp{\left[-\frac{\beta}{N}\left(\hat{T}\,+\,\hat{V}\right)\right]}\,=\,\exp{\left(-\frac{\beta}{2N}\hat{V}\right)}\exp{\left(-\frac{\beta}{N}\hat{T}\right)}\exp{\left(-\frac{\beta}{2N}\hat{V}\right)}
$$

<div style="text-align: center;">
    <span style="color: blue; font-size: 40px;"> Naive path sampling</span>
</div>

L'integrale sui cammini di Feynman descrive una singola paticella in termini di cammini $\left\{x_0,\,\dots,\,x_N\right\}$ con pesi legati alle singole matrici densità ad alta temperatura. La funzione di partizione 

$$
Z\,=\,\int dx_0 \dots dx_{N-1}\,\rho\left(x_0,\,x_1,\,\beta/N\right)\dots\rho\left(x_{N-1},\,x_0,\,\beta/N\right)
$$

può essere interpretata come una somma su tutti i cammini possibili, pesati sulle loro probabilità. Per campionare un certo cammino si sceglie casualmente un indice $k$ della catena e si valuta la mossa

$$
x_k\,\longrightarrow x_k\,+\,\delta_k
$$

con l'algoritmo di Metropolis. Il rapporto fra i pesi della nuova e della vecchia configurazione coinvolge solo i due segmenti che legano la posizione k-esima a quella precedente e a quella successiva. Solitamente un algoritmo del genere, detto di _local path sampling_ è molto lento e mostra delle problematiche a scorrelarsi dalla configurazione iniziale: i passi lunghi non vengono accettati, in quanto il peso statistico ad esso associati è praticamente nullo.